In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix, coo_matrix

from implicit.nearest_neighbours import (
    ItemItemRecommender, 
    CosineRecommender, 
    TFIDFRecommender, 
    BM25Recommender
)
import warnings
warnings.simplefilter('ignore')

In [2]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [3]:
def random_recommendation(items, n=5):
    """Случайные рекомендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [4]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Взвешенные случайные рекомендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weights. Сумма weight по всем товарам = 1
    """
    
    recs = items_weights.sample(n=n, replace=False, weights='weights')['item_id'].to_list()

    return recs

In [6]:
df = pd.read_csv('./data/retail_train.csv')

### Разобьем выборку на тренировочную и тестовую

In [7]:
test_size_weeks = 3

data_train = df[df['week_no'] < df['week_no'].max() - test_size_weeks]
data_test = df[df['week_no'] >= df['week_no'].max() - test_size_weeks]

### Подготовим датафрейм с id юзера и его реальными покупками. Сюда будем записывать рекомендации

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

### Определим веса

In [9]:
popularity = df.groupby('item_id')['sales_value'].sum().reset_index()

In [10]:
# Находим логарифм от суммы продаж товара и делим на сумму логарифмов всех сумм продаж
popularity['weights'] = (popularity['sales_value'].apply(np.log) / 
                         popularity['sales_value'].apply(np.log).replace([-np.inf], 0).sum())
popularity[popularity['weights'] < 0] = 0
items_weights = popularity[['item_id', 'weights']]

### Создадим список топ-5000 покупаемых товаров, и пометим все товары, которые в него не входят, спец-id

In [11]:
popularity = df.groupby('item_id')['quantity'].sum().reset_index().rename(columns={'quantity': 'n_sold'})
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666

### Создадим user-item матрицу

In [12]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

### Для поиска юзеров и итемов по id определим словари

In [13]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### Получим рекомендации случайным образом, взвешенным случайным и на основе самых популярных товаров

In [14]:
items = data_train.item_id.unique()
result['rnd_rec'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

In [15]:
result['weighted_rnd_rec'] = result['user_id'].apply(lambda x: 
                                                     weighted_random_recommendation(items_weights, 
                                                                                    n=5))

In [16]:
popular_recs = popularity_recommendation(data_train, n=5)
result['pop_rec'] = result['user_id'].apply(lambda x: popular_recs)

### Создадим функцию для обучения моделей и рекомендаций

In [17]:
model_list = [BM25Recommender, ItemItemRecommender, CosineRecommender, TFIDFRecommender]
column_list = ['BM25', 'itemitem', 'cosine', 'tfidf']

In [18]:
def recommend_fn(df, 
                model_name,
                user_item_matrix,
                id_to_itemid, 
                userid_to_id, 
                col_name,
                filter_items=None,
                k=5, 
                n_threads=4):
    
    model = model_name(K=k, num_threads=n_threads)
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
    df[col_name] = df['user_id'].apply(lambda user_id: [
                                       id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user_id], 
                                       user_items=sparse_user_item, 
                                       N=5, 
                                       filter_already_liked_items=False, 
                                       filter_items=filter_items, 
                                       recalculate_user=True)
                                                        ])

In [19]:
for mod, col in zip(model_list, column_list):
    recommend_fn(result, mod, user_item_matrix, id_to_itemid, userid_to_id, col)

### Трюк с K=1 и игнором помеченных товаров

In [20]:
recommend_fn(result, 
             ItemItemRecommender, 
             user_item_matrix, 
             id_to_itemid, 
             userid_to_id, 
             'mod_itemitem', 
             [itemid_to_id[6666]], 
             k=1)

### Метрики

In [21]:
for name_col in result.columns[2:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

0.0063:rnd_rec
0.0014:weighted_rnd_rec
0.0856:pop_rec
0.1255:BM25
0.1367:itemitem
0.1333:cosine
0.1391:tfidf
0.2199:mod_itemitem


___

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случайный на топ-5000 товаров

In [25]:
items = top_5000
result['rnd_rec_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [22]:
for i in range(1, 11):
    recommend_fn(result, ItemItemRecommender, user_item_matrix, id_to_itemid, userid_to_id, f'item_K{i}', k=i)    

In [26]:
for name_col in result.columns[2:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

0.0063:rnd_rec
0.0014:weighted_rnd_rec
0.0856:pop_rec
0.1255:BM25
0.1367:itemitem
0.1333:cosine
0.1391:tfidf
0.2199:mod_itemitem
0.1923:item_K1
0.192:item_K2
0.1862:item_K3
0.145:item_K4
0.1367:item_K5
0.142:item_K6
0.1449:item_K7
0.1471:item_K8
0.1486:item_K9
0.1507:item_K10
0.006:rnd_rec_5000
